In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import get_file
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import UpSampling2D


In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="BFH8qBAAyl5LcttubCWa")
project = rf.workspace("muhammad-talha-qadri-ep9rl").project("deeplab3")
dataset = project.version(1).download("coco-segmentation")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.0.76
    Uninstalling opencv-python-headless-4.8.0.76:
      Successfully uninstalled 

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Deeplab3-1 in coco-segmentation:: 100%|██████████| 29/29 [00:00<00:00, 5452.76it/s]


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory path within your Google Drive where you want to save the dataset
data_dir = "/content/Deeplab3-1"

# Define input shape and batch size
input_shape = (4000, 2250, 3)
batch_size = 4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the `set using TensorFlow Datasets
train_dataset, info = tfds.load('coco/2017', split='train', data_dir=data_dir, with_info=True)
val_dataset = tfds.load('coco/2017', split='validation', data_dir=data_dir)

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [ ]:
# Define the ASPP module
def aspp_module(inputs, filters):
    rates = [6, 12, 18]

    aspp1x1 = Conv2D(filters, (1, 1), padding='same')(inputs)
    aspp3x3_1 = Conv2D(filters, (3, 3), padding='same', dilation_rate=rates[0])(inputs)
    aspp3x3_2 = Conv2D(filters, (3, 3), padding='same', dilation_rate=rates[1])(inputs)
    aspp3x3_3 = Conv2D(filters, (3, 3), padding='same', dilation_rate=rates[2])(inputs)

    image_features = tf.reduce_mean(inputs, (1, 1), keepdims=True)
    image_features = Conv2D(filters, (1, 1), padding='same')(image_features)
    image_features = tf.image.resize(image_features, (tf.shape(inputs)[1], tf.shape(inputs)[2]))

    concatenated = Concatenate()([aspp1x1, aspp3x3_1, aspp3x3_2, aspp3x3_3, image_features])

    output = Conv2D(filters, (1, 1), padding='same')(concatenated)
    output = BatchNormalization()(output)
    output = Activation('relu')(output)

    return output

def DeeplabV3(input_shape=(4000, 2250, 3), num_classes=1):
    # Input Layer
    input_tensor = Input(shape=input_shape)

    # Backbone (ResNet)
    backbone = tf.keras.applications.ResNet50(input_tensor=input_tensor, include_top=False, weights='imagenet')

    # Atrous Spatial Pyramid Pooling (ASPP)
    x_a = aspp_module(backbone.output, 128)

    # Decoder
    x_b = Conv2D(128, (1, 1), padding='same', use_bias=False)(backbone.get_layer('conv2_block2_out').output)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    # Upsample x_a to match the spatial dimensions of x_b
    x_a = UpSampling2D(size=(8, 8), interpolation="bilinear")(x_a)

    # Resize x_b to match the spatial dimensions of x_a
    x_b = tf.image.resize(x_b, (tf.shape(x_a)[1], tf.shape(x_a)[2]), method='bilinear')

    # Concatenate ASPP and decoder features (now they have the same spatial dimensions)
    x = Concatenate()([x_a, x_b])

    x = Conv2D(256, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D(size=(6, 6), interpolation="bilinear")(x)

    # Output Layer
    output_tensor = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    # Create and compile the model
    model = Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Define input shape and number of classes
input_shape = (4000, 2250, 3)
num_classes = 1  # You can adjust this based on your dataset

# Create the DeepLabV3 model
deeplabv3_model = DeeplabV3(input_shape, num_classes)

# Display model summary
deeplabv3_model.summary()

# Compile the model
deeplabv3_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=[MeanIoU(num_classes=1)]
)

# Define callbacks for model checkpoint and early stopping
checkpoint = ModelCheckpoint("deeplabv3plus.h5", save_best_only=True)
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

# Train the model
history = deeplabv3_model.fit(
    train_dataset,
    epochs=50,  # You can adjust the number of epochs
    callbacks=[checkpoint, early_stopping]
)

# Plot training history
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['mean_io_u'], label='Mean IoU')
plt.xlabel


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_60 (InputLayer)       [(None, 4000, 2250, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 4006, 2256, 3)        0         ['input_60[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 2000, 1125, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 2000, 1125, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# Load the COCO dataset using TensorFlow Datasets
data_dir = "/content/Deeplab3-1"  # Specify the path to your dataset directory
dataset, info = tfds.load('coco/2017', split='train', data_dir=data_dir, with_info=True)

# Iterate through the dataset to get the shapes
for sample in dataset.take(5):  # Change the number to analyze more or fewer samples
    # Extract the image and mask from the sample
    image = sample['image']
    mask = sample['segmentation_mask']

    # Get the shapes
    image_shape = image.shape
    mask_shape = mask.shape

    # Print the shapes
    print("Image Shape:", image_shape)
    print("Mask Shape:", mask_shape)

    # Display the image and mask (optional)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Image')

    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('Mask')

    plt.show()

